In [1]:
import numpy as np
import pywt
import cv2

def w2d(img, mode='haar', level=1):
    imArray = img

    # Data type conversion
    # Convert to grayscale
    imArray = cv2.cvtColor(imArray, cv2.COLOR_BGR2GRAY)
    # Convert to float
    imArray = np.float32(imArray)
    imArray /= 255

    # Compute coefficients
    coeffs = pywt.wavedec2(imArray, mode, level=level)

    # Process coefficients
    coeffs_H = list(coeffs)
    coeffs_H[0] = np.zeros_like(coeffs_H[0])  # Set approximation coefficients to zeros

    # Reconstruction
    imArray_H = pywt.waverec2(coeffs_H, mode)
    imArray_H *= 255
    imArray_H = np.uint8(imArray_H)

    return imArray_H

# Example usage:
# img = cv2.imread('your_image_path.jpg')
# result = w2d(img, mode='haar', level=1)
# cv2.imshow('Original', img)
# cv2.imshow('Wavelet Transformed', result)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [2]:
import joblib
import json
import numpy as np
import base64
import cv2
from wavelet import w2d

__class_name_to_number = {}
__class_number_to_name = {}

__model = None

In [3]:
def class_number_to_name(class_num):
    return __class_number_to_name[class_num]


In [4]:

def load_saved_artifacts():
    print("loading saved artifacts...start")
    global __class_name_to_number
    global __class_number_to_name

    with open("./artifacts/class_dictionary.json", "r") as f:
        __class_name_to_number = json.load(f)
        __class_number_to_name = {v:k for k,v in __class_name_to_number.items()}

    global __model
    if __model is None:
        with open('./artifacts/saved_model.pkl', 'rb') as f:
            __model = joblib.load(f)
    print("loading saved artifacts...done")


In [5]:

def get_cv2_image_from_base64_string(b64str):
    
    encoded_data = b64str.split(',')[1]
    nparr = np.frombuffer(base64.b64decode(encoded_data), np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    return img


In [6]:
def get_cropped_image_if_2eyes(image_path, image_base64_data):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

    if image_path:
        img = cv2.imread(image_path)
    else:
        img = get_cv2_image_from_base64_string(image_base64_data)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    cropped_faces = []
    for (x,y,w,h) in faces:
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_gray)
            if len(eyes) >= 2:
                cropped_faces.append(roi_color)
    return cropped_faces

In [7]:

def classify_image(image_base64_data, file_path=None):

    imgs = get_cropped_image_if_2eyes(file_path, image_base64_data)

    result = []
    for img in imgs:
        scalled_raw_img = cv2.resize(img, (32, 32))
        img_har = w2d(img, 'db1', 5)
        scalled_img_har = cv2.resize(img_har, (32, 32))
        combined_img = np.vstack((scalled_raw_img.reshape(32 * 32 * 3, 1), scalled_img_har.reshape(32 * 32, 1)))

        len_image_array = 32*32*3 + 32*32

        final = combined_img.reshape(1,len_image_array).astype(float)
        result.append({
            'class': class_number_to_name(__model.predict(final)[0]),
            'class_probability': np.around(__model.predict_proba(final)*100,2).tolist()[0],
            'class_dictionary': __class_name_to_number
        })

    return result







In [8]:

def get_b64_test_image_for_dhoni():
    with open("base64img.txt") as f:
        return f.read()


In [9]:

if __name__ == '__main__':
    load_saved_artifacts()

    #print(classify_image(get_b64_test_image_for_dhoni(), None))
    #print(classify_image(None,'test-images/ronaldo24.png'))
    print(classify_image(None,'test-images/dhoni20.png'))
    #print(classify_image(None,'test-images/virat kohli12.png'))
    #print(classify_image(None,'test-images/dhoni20.png'))


loading saved artifacts...start
loading saved artifacts...done
[{'class': 'dhoni', 'class_probability': [56.03, 7.1, 16.18, 18.61, 2.08], 'class_dictionary': {'dhoni': 0, 'lionel messi': 1, 'maria sharapova': 2, 'ronaldo': 3, 'virat kohli': 4}}]
